<h1 align = "center">Visual Crossing API</h1>

In [1]:
import os
import sys

In [2]:
import json

In [3]:
import numpy as np
import pandas as pd

In [4]:
from tqdm import tqdm as TQ

In [5]:
import ndprdconfig
testDB = ndprdconfig.DBConnection(instance = "ap-south-1-neuralnod-dev/test")
nxprdsys = ndprdconfig.DBConnection(instance = "ap-south-1-neuralnod-dev")

engine_testdb = testDB.connect()
engine_nxprdsys = nxprdsys.connect()

In [6]:
mwCities = """
SELECT
    CityUUID
    , CityName
    , CONCAT(CityLatitude, ',', CityLongitude) AS CityLocation
FROM `dbo.mwCities`
WHERE _get_weather_data = 1
"""

mwCities = pd.read_sql(mwCities, engine_nxprdsys, index_col = "CityUUID")
mwCities["CityLocation"] = mwCities["CityLocation"].apply(eval) # sql statement format ensures tuple conversion

mwCities.sample()

,CityName,CityLocation
CityUUID,,
27FBAC74-3AC7-4B8F-B44E-2FD21FE9BEFB,Rajahmundry,"(17.00517, 81.77784)"


In [7]:
engine_nxprdsys.close()

In [8]:
sys.path.append("E:\\VisualCrossing")

<p style=" text-align:justify; "><b><code>VisualCrossing</code></b> is a simple to use package to retreive data weather data from <a href = "https://www.visualcrossing.com/weather-data">VisualCrossing</a> (website name same as that of the package). The API is so configured, that just 4-5 lines of code is sufficient to retreive data easily. The website features both historic as well as report on weather forecast. For more information, <a href = "https://www.visualcrossing.com/resources/documentation/">check documentation</a> for the same.</p>

<p style=" text-align:justify; ">This document is designed to provide a <i>quick start guide</i> for anyone who wants to test the code. Currently, the code is in <code>Development Status :: 4 - Beta</code>, thus obviously code and functionalities will be upgraded with new features. <b>Note:</b> minimal documentation is currently provided, and will be updated in time.</p>

Get started by `pip install VisualCrossing`, and then import the package as:

In [9]:
import VisualCrossing as vc

# get installed version
# print(vc.__version__)

In [16]:
help(vc.API) # check documentation

Help on class API in module VisualCrossing.api:

class API(base)
 |  API(start: str, locations: list, key: str, **kwargs) -> None
 |  
 |  A python wrapper to fetch weather data from VisualCrossing
 |  
 |  The function is devised keeping in mind - minimal coding approach, and
 |  recurrent job-scheduling for any person. All the keyword arguments that
 |  the API accepts are same as that available in the Visual Crossing API
 |  Documentation (check README for more information). The API is configured
 |  with default values, which is easier to understand and retreive. However,
 |  the data requires certain required arguments defined below. The same can
 |  also be obtained from `config.json` file, but is not recommended.
 |  
 |  :type  start: str or dt.date
 |  :param start: The starting and ending data to fetch the data from the
 |      API. The date is to be either in `str("%Y-%m-%d")` format or an
 |      instance of the `dt.date()` class.
 |      
 |  :type  locations: list
 |  :pa

The API requires three minimal options:
1. Date for which the data is required, in `YYYY-MM-DD` format,
2. API Key which can be obtained from [website](https://www.visualcrossing.com/weather-api), and
3. Location for which data is required. The location can be of the following types:
   * Name of a single place, like `"india"`,
   * Coordinates of a place (if data for the coordinate is not available, then nearest location data is returned), this should be in `"latitude,longitude"` format,
   * To retreive location for multiple location, pass the data like:
   ```python
    ["india", "tokyo"] # either passing names,
    ["20.593683,78.962883", "35.689487,139.691711"] # passing list/tuple of coordinates,
    ("india", "35.689487,139.691711") # or even a combination of name or coordinate
   ```
   * _you can also use US ZIP Codes_.

Location is also available by `(lat, lon)` pairs, for which you can also use [this website](https://www.latlong.net/) to find location easily.

In [10]:
locations = {idx : f"{row.CityLocation[0]},{row.CityLocation[1]}" for idx, row in mwCities.iterrows()}
# api = vc.API("2021-09-25", APIKey = os.getenv("__api_key__"), location = locations, contentType = "dataframe") # default parameters are set from `config.json`

In [11]:
api = vc.API(start = "1975-01-01", locations = locations.values(), key = "", end = "1975-12-31")

In [12]:
responses = api.response()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162/162 [04:59<00:00,  1.85s/it]

Total Query Cost: 1,084,627


In [13]:
def exists(station_id : str, engine : object) -> bool:
    # check if a station exists
    statement = f"SELECT * FROM `dbo.mwVCWeatherStations` WHERE VCWeatherStationID = '{station_id}';"
    records = engine.execute(statement).fetchall()
    return True if records else False

In [14]:
for response in TQ(responses):
    try:
        for station, value in response["stations"].items():
            insert_statement = "INSERT INTO `dbo.mwVCWeatherStations` VALUES ('{VCWeatherStationID}', '{StationName}', {StationLatitude:.6f}, {StationLongitude:.6f}, {StationQuality:.2f}, {StationContribution:.2f});"
            insert_statement = insert_statement.format(
                VCWeatherStationID = station,
                StationName = value["name"],
                StationLatitude = value["latitude"],
                StationLongitude = value["longitude"],
                StationQuality = value["quality"],
                StationContribution = value["contribution"]
            )

            if not exists(station, engine = engine_testdb):
                engine_testdb.execute(insert_statement)
    except Exception as err:
        pass # print("Failed to Fetch Stations for Resolved Address = ", response["resolvedAddress"])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162/162 [00:08<00:00, 18.79it/s]


In [15]:
rename_final_columns = {
    "datetime" : "EffectiveDate",
    "tempmax" : "TempMax",
    "tempmin" : "TempMin",
    "temp" : "TempAvg",
    "feelslikemax" : "FeelsLikeMax",
    "feelslikemin" : "FeelsLikeMin",
    "feelslike" : "FeelsLikeAvg",
    "dew" : "DewTemp",
    "humidity" : "HumidityPercent",
    "precip" : "PrecipAmt",
    "precipprob" : "PrecipProb",
    "precipcover" : "PrecipCover",
    "preciptype" : "PrecipType",
    "snow" : "SnowCover",
    "snowdepth" : "SnowDepth",
    "windgust" : "WindGust",
    "windspeed" : "WindSpeed",
    "winddir" : "WindDir",
    "pressure" : "AtmPressure",
    "cloudcover" : "CloudCoverage",
    "visibility" : "VisibilityDist",
    "solarradiation" : "SolarRadiation",
    "solarenergy" : "SolarEnergy",
    "sunrise" : "SunRise",
    "sunset" : "SunSet",
    "moonphase" : "MoonPhase",
    "conditions" : "WeatherCond",
    "description" : "WeatherDesc",
    "icon" : "WeatherIcon",
    "stations" : "_stations",
    "source" : "_source",
}

In [16]:
tblDailyWeather = []
for response, (CityUUID, _) in TQ(zip(responses, locations.items()), total = len(responses)):
    ResolvedLocation = response["resolvedAddress"]
    tblDailyWeather_ = pd.DataFrame(response["days"])
    
    for existing_col_name, final_col_name in rename_final_columns.items():
        try:
            tblDailyWeather_.rename(columns = {existing_col_name : final_col_name}, inplace = True)
        except:
            pass
        
    tblDailyWeather_ = tblDailyWeather_[[col for col in tblDailyWeather_.columns if col in list(rename_final_columns.values())]]
    tblDailyWeather_["CityUUID"] = CityUUID
    tblDailyWeather_["ResolvedLocation"] = ResolvedLocation
    
    if "_stations" in tblDailyWeather_.columns:
        tblDailyWeather_["_stations"] = tblDailyWeather_["_stations"].apply(lambda x : ",".join(x) if x else None)
    else:
        tblDailyWeather_["_stations"] = None
        
    if "PrecipType" in tblDailyWeather_.columns:
        tblDailyWeather_["PrecipType"] = tblDailyWeather_["PrecipType"].apply(lambda x : ",".join(x) if x else None)
    else:
        tblDailyWeather_["PrecipType"] = None
    
    tblDailyWeather.append(tblDailyWeather_)

tblDailyWeather = pd.concat(tblDailyWeather)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162/162 [00:01<00:00, 86.73it/s]


In [17]:
tblDailyWeather["SunRise"] = pd.to_datetime(tblDailyWeather[["EffectiveDate", "SunRise"]].apply(lambda x : f"{x[0]} {x[1]}", axis = 1), format = "%Y-%m-%d")
tblDailyWeather["SunSet"] = pd.to_datetime(tblDailyWeather[["EffectiveDate", "SunSet"]].apply(lambda x : f"{x[0]} {x[1]}", axis = 1), format = "%Y-%m-%d")

In [18]:
for column in TQ(tblDailyWeather.columns):
    tblDailyWeather[column] = tblDailyWeather[column].apply(lambda x : np.nan if str(x).strip().replace("  ", " ") == "" else x)
    tblDailyWeather[column].fillna(value = np.nan, inplace = True)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:01<00:00, 17.23it/s]


In [19]:
tblDailyWeather.to_sql("tblDailyWeather", engine_testdb, index = False, if_exists = "append")

59130

In [20]:
with open("responses.json", "w") as f:
    json.dump(responses, f, indent = 2)